## XGB, LightGBM ,RandomForestClassifier objective

In [1]:
import numpy as np 
import optuna

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import datasets
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
import psutil
import time 
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
asd = pd.read_csv('/home/jupyter/climate/data/TEST_with_파생변수0802.csv')

In [7]:
asd.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'yyyymmdd', 'add', 'sex', 'frequency',
       'avg_ps', 'max_ps', 'min_ps', 'avg_td', 'max_td', 'max_td_hrmt',
       'min_td', 'min_td_hrmt', 'avg_ta_x', 'max_ta_x', 'min_ta_x', 'so2',
       'co', 'o3', 'no2', 'pm10', 'avg_ta_y', 'max_ta_y', 'min_ta_y',
       'min_mrng_ta', 'max_dytm_ta', 'min_nght_ta', 'holiday',
       'over_SO2_WHO_crit', 'over_PM10_WHO_crit', 'over_O3_WHO_crit', '한파',
       '폭염', 'season_fall', 'season_spring', 'season_summer', 'season_winter',
       'weekday_Fri', 'weekday_Mon', 'weekday_Sat', 'weekday_Sun',
       'weekday_Thu', 'weekday_Tue', 'weekday_Wed', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12', 'min_mrng_ta_lag_1',
       'min_mrng_ta_lag_2', 'min_mrng_ta_lag_3', 'min_mrng_ta_lag_4',
       'min_mrng_ta_lag_5', 'min_mrng_ta_lag_6', 'min_mrng_ta_lag_7',
       'min_ps_lag_1', 'min_ps_lag_2', 'min_ps_lag_3', 'min_

1.XGBoost 최적 하이퍼 파라미터 

In [14]:
def objective_xgb(trial):

    params = {
        "objective": "multi:softprob",
        "eval_metric":'mlogloss',
        "booster": 'gbtree',
        'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시 
        "verbosity": 0,
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),     
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
    }
    
    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
    
    f1score=[]
    
    for train_index, test_index in str_kf.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        model = xgb.XGBClassifier(**params, random_state = 1234, use_label_encoder = False)
        model.fit(x_train,y_train)
        
        y_pred=model.predict(x_val)
        f1score.append(f1_score(y_val,y_pred))
        total=np.mean(f1score)
        
    
    return total

2. SVC (linear Kernal) 하이퍼 파라미터 

In [4]:
import sklearn.svm

In [15]:
def objective_svc(trial):
    svc_c = trial.suggest_float("C", 1e-10, 1e10, log=True)
    model = sklearn.svm.SVC(C=svc_c, gamma="auto")
    
    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
    f1score=[]
    
    for train_index, test_index in str_kf.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        model.fit(x_train,y_train)
        
        y_pred=model.predict(x_val)
        f1score.append(f1_score(y_val,y_pred))
        total=np.mean(f1score)
    
    return total

3. LightGBM 파이퍼 파라미터 

In [7]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
def objective_lgbm(trial):
    
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
        'max_depth': trial.suggest_int('max_depth', 1, 10, step=1, log=False), 
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 8, 1024, step=1, log=True), 
        'objective': 'multiclass', 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=1, log=False), 
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'random_state': 0
    }
    
    
    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
    f1score=[]
    
    for train_index, test_index in str_kf.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        model=lgb_clf= lgb.LGBMClassifier(**params)
        model.fit(x_train,y_train)
        
        y_pred=model.predict(x_val)
        f1score.append(f1_score(y_val,y_pred))
        total=np.mean(f1score)
    
    return total

4. RandomForestClassifier 하이퍼 파라미터 튜닝 

In [16]:
def objective_rfc(trial):
    
    params={
        
        'max_depth' : trial.suggest_int('max_depth', 1, 20),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 2, 1000),
        'n_estimators' : trial.suggest_int('n_estimators', 100, 500),
        'ccp_alpha' : trial.suggest_float('ccp_alpha', 0.0, 200, log=True), 
        'bootstrap' : True,
        'oob_score' : True,
        'random_state': 6,
        'warm_start' : True
    }
    
    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
    f1score=[]
    
    for train_index, test_index in str_kf.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        model=RandomForestClassifier(**params)
        model.fit(x_train,y_train)
        
        y_pred=model.predict(x_val)
        f1score.append(f1_score(y_val,y_pred))
        total=np.mean(f1score)
    
    return total
    

5. isolation forest 하이퍼 파라미터 튜닝 

In [1]:
def objective_if(trial):
    # global X_train
    # global X_test
    
    params={
        
        'n_estimators' :trial.suggest_int('n_estimators', 1, 200),
        'max_samples' : trial.suggest_uniform('max_samples', 0.0, 1.0),
        'contamination' : trial.suggest_loguniform('contamination', 1e-6, 1e-1),
        'max_features' : trial.suggest_uniform('max_features', 0.0, 1.0),
        'bootstrap' : trial.suggest_categorical('bootstrap', [True, False]) }
    
    # 아래 잘못된 변수명 있으면 고치기 
    
    clf = IsolationForest(**params)
    clf.fit(X_train) 
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    y_pred_test = pd.DataFrame(y_pred_test)
    y_pred = []
    
    
    for i in range(len(y_pred_test)):
        if y_pred_test.iloc[i,0] == 1 :
            y_pred_test.iloc[i,0] = 0
        else:
            y_pred_test.iloc[i,0] = 1
        
    f1 = f1_score(y_test, y_pred_test,average='macro')
    
    return f1
    